# Convolutional Neural Network

### Importing the libraries

In [35]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [36]:
tf.__version__

'2.6.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [37]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [38]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [39]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [40]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [41]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [42]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a third convolutional layer

In [43]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='sigmoid'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [44]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [45]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [46]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [47]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [48]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 62s 245ms/step - loss: 0.6967 - accuracy: 0.5326 - val_loss: 0.6831 - val_accuracy: 0.5810
Epoch 2/25
250/250 [==============================] - 61s 244ms/step - loss: 0.6825 - accuracy: 0.5699 - val_loss: 0.6766 - val_accuracy: 0.5985
Epoch 3/25
250/250 [==============================] - 62s 248ms/step - loss: 0.6710 - accuracy: 0.5881 - val_loss: 0.6667 - val_accuracy: 0.6090
Epoch 4/25
250/250 [==============================] - 62s 248ms/step - loss: 0.6503 - accuracy: 0.6124 - val_loss: 0.6262 - val_accuracy: 0.6480
Epoch 5/25
250/250 [==============================] - 67s 269ms/step - loss: 0.6091 - accuracy: 0.6604 - val_loss: 0.5812 - val_accuracy: 0.6975
Epoch 6/25
250/250 [==============================] - 81s 324ms/step - loss: 0.5820 - accuracy: 0.6884 - val_loss: 0.6043 - val_accuracy: 0.6830
Epoch 7/25
250/250 [==============================] - 76s 305ms/step - loss: 0.5649 - accuracy: 0.7044 - val_loss: 0.5585 - val_ac

## Part 4 - Making a single prediction

In [49]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [50]:
print(prediction)

cat


## Fim